### Algorithm to Approximate Stabilizer Rank
* https://arxiv.org/abs/1711.07848 - On the Geometry of Stabilizer States
* $\underline{\text{First Approach - Approximating an arbitrary quantum state with a single stabilizer state (Section 4.1)}}$
    * Outline of approach:
        * Begin with arbitrary state - 
        $$\ket{\psi} = \sum_{i = 1}^{k} \alpha_i \ket{s_i}$$
        * Start at stabilizer state $\ket{s_m}$ such that it has largest $|\alpha_i|$
        * At each iteration, evaluate $N(\ket{s_m})$ (nearest neighbors of $\ket{s_m}$) by computing 
        $$\max_{\ket{\phi} \in N(\ket{s_m})} |\bra{\phi}\ket{\psi}|$$